In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
from datetime import datetime, timedelta
import pymysql
from selenium.webdriver.support.ui import WebDriverWait

In [20]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')

chrome = webdriver.Chrome('./chromedriver')

search_date = datetime.strftime(datetime.now() , '%Y-%m-%d')
# 資料庫設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3307, 
    "user": "root",
    "password": "0531",
    "db": "python",
    "charset": "utf8"
}
conn = pymysql.connect(**db_settings)

In [26]:

tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    #加入台幣的cookie，預設為美金
    chrome.delete_cookie("i18n-prefs")    
    chrome.add_cookie({"name":"i18n-prefs","value": "TWD"})
    chrome.refresh()
    try:
        soup = BeautifulSoup(chrome.page_source,'lxml')
        amazon_card=soup.select('div[class="a-section a-spacing-medium"] > div[class="sg-row"] > div[class="sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 sg-col"]')
        if len(amazon_card) == 0:
            amazon_card = soup.find_all(class_="s-expand-height s-include-content-margin s-border-bottom s-latency-cf-section")
        with conn.cursor() as cursor:
            for datas in amazon_card:                
                try:
                    brand_item = datas.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).text        
                except:
                    brand_item = datas.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'}).text 
                try:                    
                    price = int(datas.find('span',{'class':'a-price-whole'}).text.strip('.').replace(',',''))
                except:
                    price_url = datas.find('a',{'class':'a-link-normal a-text-normal'}).get('href')    
                    chrome.get('https://www.amazon.com'+price_url)        
                    popo = BeautifulSoup(chrome.page_source,'lxml')
                    
                    try:
                        obj1 = popo.select('a[class="a-link-normal"] > span[class="a-size-base a-color-price"]')[0].text
                        p = re.search(r'(\d+),(\d+)',obj1)
                        price = int(p.group().replace(',',''))
                    except: 
                        price = 0                   
                matchObj = re.split(r'\s', brand_item, re.M|re.I)    
                insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s,%s,%s,%s);"
                cursor.execute(insert,(matchObj[0],brand_item,price,i,search_date))
                print(insert%(matchObj[0],brand_item,price,i,search_date))
    except Exception as ex:
        print(ex)
conn.commit() 
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( MSI, MSI Gaming Radeon RX GDRR5 DirectX 12 VR Ready CFX 顯示卡,8600,1,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( MSI微星, MSI微星 Gaming GeForce RTX 3070 8GB GDRR6 256-Bit HDMI/DP TORX 風扇 3.0 Ampere Architecture OC 顯示卡(RTX 3070 Ventus 3X OC),25229,1,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte 技嘉 Radeon RX 580 電競 8GB 顯示卡 GV-RX580GAMING-8GD REV2.0,8572,1,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte 技嘉 Geforce GTX 1050 Ti 顯示卡 內建 4GB 記憶體 搭載風之力風扇 繪圖晶片（GV-N105TWF2OC-4GD）,7167,1,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte 技嘉 GV-N1030OC-2GI Nvidia GeForce GT 1030 OC 2G 顯示卡,2551,1,2021-01-04);
INSERT 

INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( ZOTAC, ZOTAC Gaming GeForce RTX 3060 Ti Twin Edge 8GB GDDR6 256-bit 14 Gbps PCIE 4.0 遊戲顯示卡,IceStorm 2.0 進階冷卻,主動式風扇控制,ZT-A30610E-10M,0,2,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( ASUS, ASUS 華碩 ROG Strix AMD Radeon RX 5600 XT TOP Edition 遊戲顯示卡 (PCIe 4.0、6GB GDDR6 記憶體、HDMI、DisplayPort、1080p 遊戲、Axial-tech 風扇設計、自動至尊、金屬背板),15480,2,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Asus, Asus PH-GT1030-O2G GeForce GT 1030 2GB Phoenix Fan OC Edition HDMI DVI 顯示卡,2474,2,2021-01-04);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte GeForce GT 710 1GB Graphic Cards and Support PCI Express 2.0 X8 Bus Interface. 內部記憶體: 1 GB Graphic Cards GV-N710D5-1GL REV2.0 藍色,1031,2,2021-01-04);
INSERT INTO `price_comparison` (`brand`,

In [ ]:
conn.commit() 
conn.close()
cursor.close()
chrome.close() 